In [ ]:
import json

from stravalib import Client

In [3]:
access_token = None
activity_id = None

In [ ]:
client = Client(access_token)

# Retrieve Strava athlete
athlete_response = client.get_athlete()
athlete = athlete_response.to_dict()

# Retrieve Strava activity details
activity_detail_response = client.get_activity(activity_id)
activity_detail = activity_detail_response.to_dict()

# Retrieve Strava activity streams
STRAVA_STREAM_TYPES = [
    'time', 'latlng', 'distance', 'altitude', 'velocity_smooth', 'heartrate',
    'cadence', 'watts', 'temp', 'moving', 'grade_smooth'
]
activity_streams_response = client.get_activity_streams(
    activity_id=activity_id,
    types=STRAVA_STREAM_TYPES,
    series_type='time')
activity_streams = {key: val.data for key, val in activity_streams_response.items()}

In [ ]:
print(f'athlete firstname: {athlete["firstname"]}')
print(f'activity_detail: {activity_detail["external_id"]}')
print(f'activity_streams: {activity_streams["altitude"]}')